# Creating and Publishing a Python Library

Early in the course we showed how easy it is to download packages from PyPI (Python Package Index), the official repository for all Python pacakges. This class will discuss process of creating and publishing a library to the outer world, which is a bit more complicated than downloading a package, but not by much.

## Naive Solution

The "dumbest" way to do it is to send a `.py` file to someone by mail. If the receiver has a Python interpreter installed, he can just run the file by writing `python new_file.py` in his command line. This has a few short-comings:

1. The script was written in a specific Python version. You have to make sure that your receiver has the same Python version.
2. If it's more than one script, you have to bundle scripts together and make sure they remain in the same directory when run.
3. Nearly all scripts have dependencies. There's no way to verify that the receiver truly has all the dependencies and is using the same version of dependencies that you're using.
4. If the original script is using code not written in Python, like Cython code or other C\C++ modules, you need to build these scripts in place - you can't rely on them working on the target computer.

These issues, along with the relative ease with which one can publish Python code online, support a more robust approach to code sharing.

## Python Libraries from the Ground Up

There are a few "layers" for Python projects and libraries. The basic layers one can encounter in each Python (and software) project are source-code organization and version control. This tutorial will guide you through the process of creating a full-blown project, including PyPI publishing and test-suite support. If you're aiming at something lighter you can just drop some of the components used here.

Features of this project include:
1. GitHub integration
2. Continuous Integration via GitHub Actions
3. Semantic versioning support
5. Easy PyPI uploading
6. Automatic docs generation.

Many of these features can be generated using [`cookiecutter`](https://cookiecutter.readthedocs.io/en/1.7.2/index.html) but we'll avoid that for now. Once we understand what's going on under the hood we might prefer using `cookiecutter` instead of going through these manual steps. This guide is heavily based on a [famous blog post](https://hynek.me/articles/sharing-your-labor-of-love-pypi-quick-and-dirty/).

### New project setup - scaffolding

I'll be assuming that we're creating a new project, but these steps can easily be adapted if you wish to convert an existing project.

1. Make a new directory with the project's name. We'll use "Parse Stuff" as our project name here, so this directory may be called `ParseStuff`, `parseStuff`, `parse_stuff` or really any other name.
2. Open it in VSCode using "Open Folder...".
3. Generate several basic files that we'll fill in later:
    * Create `README.md` and write inside a single line describing the project.
    * Create an empty `CHANGELOG.md` file.
    * Create `.gitignore` and copy-paste the content from [here](https://github.com/github/gitignore/blob/master/Python.gitignore).
    * Create a `src` folder, and inside it create a folder with the name of your project **in snake case**. In our case, the folder is `src/parse_stuff`. Inside that folder create a `__init__.py` file.
    * Inside that file fill in the following:
    ```
__version__ = "0.1.0"
__title__ = "Parse Stuff"
__description__ = "Parse Stuff With Ease"
__url__ = <GITHUB LINK>
__doc__ = __description__ + " <" + __url__ + ">"
__author__ = <YOUR NAME>
__email__ = <YOUR EMAIL>
__license__ = "MIT"
    ```
    The "version" string means that the "major" version is 0, the "minor" version is 1 and the "patch" version is 0. This follows [semantic versioning](https://semver.org/), which is the simplest and clearest versioning scheme available. 
    * Create a file named `LICENSE` (no suffix needed) and copy the content from [here](https://choosealicense.com/licenses/mit/). This is the MIT license which gives other people permission to basically do whatever they want with your code. Make sure to replace the "year" and "name" placeholders.
    * Create a folder named `tests` and inside it create an empty `__init__.py` file.
    

4. Now that we have a couple of basic files in our hands, create a new git repository in that folder using VSCode's "Initial repo" button.
5. In the same git menu, add all files (using the "+" button), write a commit message ("Initial commit" will do) and commit by clicking the "V".
6. Go to your GitHub account and create a new project with a name matching the name of the folder in your system (i.e. `ParseStuff`). You don't need a license or a .gitignore file, an empty project will do just fine.
7. Copy the link to the repo (the one ending with `.git`) and go back to VSCode. Press Ctrl+Shift+P and write "Git: Add Remote". Paste the link, insert "`origin`" as the repo name and confirm other dialogue boxes.
8. If everything worked then you should be able to push by clicking the "..." icon and choosing the "Push" option. Make sure your code was indeed uploaded to the repo.

### Environment and dependencies

We have a folder but the project really isn't a project quite yet, and we also didn't do anything quite new yet. Let's do some Python-related work:

1. Create a new environment and activate it: `conda create -n parsestuff python=3.8`. Once it's done write `conda activate parsestuff`.
2. Install several key dependencies. You can add to this list any dependencies you know you'll have for the actual code: `pip install pep517 twine pytest mypy black flake8 pylint`
3. Add configuration files. These files configure the behavior of the tools we just installed, so it's OK if you don't completely understand every written word here.
    * Generate a file called `pyproject.toml` and write inside the following:
    ```
[tool.black]
line-length = 88
target-version = ['py36', 'py37', 'py38']
include = '\.pyi?$'
exclude = '''
/(
    \.eggs
  | \.git
  | \.hg
  | \.mypy_cache
  | \.tox
  | \.venv
  | _build
  | buck-out
  | build
  | dist
)/
'''
[build-system]
requires = ["setuptools>=41.0", "wheel"]
build-backend = "setuptools.build_meta"
```
    * Generate a file called `.flake8` and write inside the following:
    ```
[flake8]
ignore = E203, E266, E501, W503
# line length is intentionally set to 80 here because black uses Bugbear
# See https://github.com/psf/black/blob/master/README.md#line-length for more details
max-line-length = 88
max-complexity = 18
select = B,C,E,F,W,T4,B9
```

    * Generate a file called `MANIFEST.in` and write inside the following:
    ```
include LICENSE *.rst *.toml *.yml *.yaml
# Tests
recursive-include tests *.py
# Documentation
recursive-include docs *.png
recursive-include docs *.svg
recursive-include docs *.py
recursive-include docs *.rst
prune docs/_build
```
    * Generate a file called `requirements.txt` and write inside the following:
    ```
--index-url https://pypi.python.org/simple/
-e .
```
    * Generate a file called `.pypirc` and write inside the following:
    ```
[distutils]
index-servers=
    pypi
    test
[test]
repository = https://test.pypi.org/legacy/
username = <your test user name goes here>
[pypi]
username = __token__
```
    * Write `pip install mkdocs` in your console, followed by a `mkdocs new .` This should generate an `mkdocs.yml` file and a `docs` folder with an `index.md` in it. At the end of the `mkdocs.yml` file you should add `theme: readthedocs`. We'll leave everything empty right now, and we'll come back to this later.
4. Add a `setup.py` file with the following content:

In [3]:
import codecs
import os
import re

from setuptools import setup, find_packages


###################################################################

NAME = "parse_stuff"
PACKAGES = find_packages(where="src")
META_PATH = os.path.join("src", "parse_stuff", "__init__.py")
KEYWORDS = ["class", "attribute", "boilerplate"]
CLASSIFIERS = [
    "Development Status :: 3 - Alpha",
    "Intended Audience :: Developers",
    "Natural Language :: English",
    "License :: OSI Approved :: MIT License",
    "Operating System :: OS Independent",
    "Programming Language :: Python",
    "Programming Language :: Python :: 3",
    "Programming Language :: Python :: 3.6",
    "Programming Language :: Python :: 3.7",
    "Programming Language :: Python :: 3.8",
    "Programming Language :: Python :: Implementation :: CPython",
]

INSTALL_REQUIRES = [
    'pep517',
    'twine',
    'pytest',
    'mypy',
    'black',
    'flake8',
    'pylint',
]

###################################################################

HERE = os.path.abspath(os.path.dirname(__file__))


def read(*parts):
    """
    Build an absolute path from *parts* and and return the contents of the
    resulting file.  Assume UTF-8 encoding.
    """
    with codecs.open(os.path.join(HERE, *parts), "rb", "utf-8") as f:
        return f.read()


META_FILE = read(META_PATH)


def find_meta(meta):
    """
    Extract __*meta*__ from META_FILE.
    """
    meta_match = re.search(
        r"^__{meta}__ = ['\"]([^'\"]*)['\"]".format(meta=meta),
        META_FILE, re.M
    )
    if meta_match:
        return meta_match.group(1)
    raise RuntimeError(f"Unable to find __{meta}__ string.")


if __name__ == "__main__":
    setup(
        name=NAME,
        description=find_meta("description"),
        license=find_meta("license"),
        url=find_meta("url"),
        version=find_meta("version"),
        author=find_meta("author"),
        author_email=find_meta("email"),
        maintainer=find_meta("author"),
        maintainer_email=find_meta("email"),
        keywords=KEYWORDS,
        long_description=read("README.md"),
        packages=PACKAGES,
        package_dir={"": "src"},
        zip_safe=False,
        classifiers=CLASSIFIERS,
        install_requires=INSTALL_REQUIRES,
)

NameError: name '__file__' is not defined

Before we go any further let's recap our latest steps: First, we're defining configuration files for some of the tools we'll be using, like `black` and `flake8`. These are definitely not a must, but they're nice to have. Then we added an eerie-looking `MANIFEST.in` file which contains a bunch of incantations. This file define what will be packaged with our code besides the actual code, but I won't go into  any further details. Lastly we added a `setup.py` file which is basically a configuration\metadata file that defines how will our package be packaged. It's a standard Python file but it has a pretty weird syntax since it doesn't deal with the code, it deals with code generation. You can see a few fields containing the name of the package and other things related to it, but it's mostly cryptic and shouldn't be of much concern at the moment. In general you should be able to copy paste this file from one project to the other, and only change the stuff that's inside the `###` block.

### Build and publish!

Before doing more advanced stuff, let's see if it builds. Building a package means generating a file that can be more easily shared with others, since this file (or files) contain metadata regarding our dependencies, Python version, and more. Every time we build we first have to remove the `dist` folder into which we'll build our package. Since this is the first time we're building, we don't have this folder yet, but don't forget to do it next time. To build, write in the command line (when you're at the top project directory):

`python -m pep517.build .`

If everything was successful, you should see a new `dist` and two files there - a `wheel` file and a `tar.gz` file. These files can be shared and installed with `pip` on other computers, but the package can't be installed from the web yet. To do that we'll have to upload them to PyPI. Before doing that it's advised that you should make a new virtual environment and see if they can be installled.

Assuming installation was successful, we can try to upload them to PyPI. We'll first upload it to a test ('staging') server, and then to the real PyPI. To create a username in the testing server, go [here](https://test.pypi.org/) and create a new username. Fill in this username in the previously generated `.pypirc` file. Now we'll use `twine` to upload it to the staging area:

`twine upload -r test dist/parse_stuff*`

If the upload was reported as successful, try to `pip`-install it (again, in a new environment) with the following command:
`pip install -i https://testpypi.python.org/pypi parse_stuff`

If it can be imported, and your page online in the testing area looks fine, then we're ready to upload it to the "real" PyPI. Write:

`twine upload -r pypi dist/parse_stuff*` 

and witness the wonder. Easy, right? :)

### Documentation and ReadTheDocs

Documenting your project is extremely important. Besides having comments in your code and helpful docstrings, you sometimes also want more high-level material regarding installation and usage of your pacakge. This info should go in the `docs` folder that was generated earlier. The idea is to build a website from markdown (`.md`) files. Please consult the [MkDocs website](https://www.mkdocs.org/) for more details.

If our project is on PyPI, we also want our documentation to have a website. ReadTheDocs is a website that happily hosts the documentation for your code. It will automatically fetch whatever's in your `docs` folder and display it in a nice-looking website.

1. Create an account [there](https://readthedocs.org) and log into it.

2. Go to your dashboard and Import a repository from the "My Projects" pulldown.

3. Inside GitHub, select Settings -> Webhooks and turn on the ReadTheDocs hook.

Your docs will update online every time you update them and push the changes to GitHub. They're probably currently quite dull, but you should add more documents to the `docs` folder sooner rather than later.

### Continuous Integration

CI is used to automatically test your code whenever you push something to GitHub. It works by pulling the latest commit from your GitHub into its servers. It then builds an environment identical to the one you have in your computer, installs your package and runs the tests for that package. Lastly it generates a detailed error report if one or more of your tests failed. This section is - again - based on a [famous blog post](https://hynek.me/articles/python-github-actions/).

The easiest CI tool available today is GitHub actions. It's integrated inside GitHub and allows you monitor the execution easily. To set it up, go to the "Actions" tab in your GH repo and click "Skip this and setup a workflow yourself". Delete the template and paste this instead:

```yaml
---
name: CI

on:
  push:
    branches: ["master"]
  pull_request:
    branches: ["master"]

jobs:
  tests:
    name: "Python ${{ matrix.python-version }}"
    runs-on: "ubuntu-latest"
    env:
      USING_COVERAGE: '3.6,3.8'

    strategy:
      matrix:
        python-version: ["3.6", "3.7", "3.8"]

    steps:
      - uses: "actions/checkout@v2"
      - uses: "actions/setup-python@v1"
        with:
          python-version: "${{ matrix.python-version }}"
      - name: "Install dependencies"
        run: |
          set -xe
          python -VV
          python -m site
          python -m pip install --upgrade pip setuptools wheel
          python -m pip install --upgrade pytest

      - name: "Run pytest"
        run: "python -m pytest"`
```

To conclude this part, "...click the green “Start commit” button in the top right and make sure you select the “Create a new branch for this commit and start a pull request.” radio button. Give the branch a memorable name (e.g. github-actions) and subsequently click “Create pull request”."

These changes are only updated in our web repo. We'll need to pull them in order to see them locally.